# Building Production Ready RAG Pipeline

In this notebook you will learn to build a Production ready RAG Pipeline on `Attention is All You Need` paper. We will use `Sentence Window Index` to build a basic RAG pipeline and iterate over different parameters to make it production ready.

Following are the steps involved:

1. Download Data
2. Load Data
3. Build Evaluation Dataset.
4. Download `RagEvaluatorPack`.
5. Define LLM, Embedding Model.
6. Build RAG with `Sentence Window` approach.
7. Evaluate RAG Pipeline.
8. Create functions to build index, evaluate.
9. Tune different parameters to improve metrics and make it production ready.

## Download `Attention is all you need` paper.

In [11]:
# attach to the same event-loop
import nest_asyncio

nest_asyncio.apply()

In [1]:
!mkdir './data'
!wget --user-agent="Mozilla" "https://arxiv.org/pdf/1706.03762.pdf" -O "./data/attention_is_all_you_need.pdf"

--2024-03-31 14:47:14--  https://arxiv.org/pdf/1706.03762.pdf
Resolving arxiv.org (arxiv.org)... 151.101.195.42, 151.101.131.42, 151.101.3.42, ...
Connecting to arxiv.org (arxiv.org)|151.101.195.42|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2215244 (2.1M) [application/pdf]
Saving to: ‘./data/attention_is_all_you_need.pdf’

./data/attention_is 100%[===================>]   2.11M   959KB/s    in 2.3s    

2024-03-31 14:47:16 (959 KB/s) - ‘./data/attention_is_all_you_need.pdf’ saved [2215244/2215244]



## Set `OpenAI` keys.

In [2]:
import os 
from dotenv import load_dotenv, find_dotenv

In [3]:
load_dotenv('/home/santhosh/Projects/courses/Pinnacle/.env')

True

In [4]:
OPENAI_API_KEY = os.environ['OPENAI_API_KEY']

## Load Data.

We will use first 10 pages and skip paper references in the paper.

In [6]:
from llama_index.core import SimpleDirectoryReader

data = SimpleDirectoryReader('./data/').load_data()

documents = data[:1]

## Generate Evaluation dataset using `RagDatasetGenerator` and `GPT-4`

In [9]:
from llama_index.llms.openai import OpenAI
from llama_index.core.llama_dataset.generator import RagDatasetGenerator

In [12]:
gpt4 = OpenAI(model='gpt-4-0125-preview', temperature=0.1)

dataset_generator = RagDatasetGenerator.from_documents(
    documents,
    llm=gpt4,
    num_questions_per_chunk=2,
    show_progress=True,
)

eval_dataset = dataset_generator.generate_dataset_from_nodes()

Parsing nodes:   0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 2/2 [00:18<00:00,  9.01s/it]


## Import `RagEvaluatorPack` for evaluation.

In [13]:
from llama_index.packs.rag_evaluator import RagEvaluatorPack

## Define LLM

In [14]:
from llama_index.llms.openai import OpenAI

llm = OpenAI(model="gpt-3.5-turbo", temperature=0.1)

## Define Embedding Model

In [15]:
embed_model = "local:BAAI/bge-small-en-v1.5"

## Build RAG pipeline with `SentenceWindow`

In [16]:
from llama_index.core.node_parser import SentenceWindowNodeParser

# create the sentence window node parser w/ default settings
node_parser = SentenceWindowNodeParser.from_defaults(
    window_size=1,
    window_metadata_key="window",
    original_text_metadata_key="original_text",
)

In [17]:
from llama_index.core import Document

document = Document(text="\n\n".join([doc.text for doc in documents]))

In [19]:
from llama_index.core import VectorStoreIndex

sentence_index = VectorStoreIndex.from_documents(
    [document], embed_model=embed_model, llm=llm)

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/133M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [20]:
from llama_index.core.indices.postprocessor import SentenceTransformerRerank

rerank = SentenceTransformerRerank(
    top_n=2, model="BAAI/bge-reranker-base"
)

config.json:   0%|          | 0.00/799 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/279 [00:00<?, ?B/s]

/home/santhosh/anaconda3/lib/python3.11/json/decoder.py:353: RuntimeWarning: coroutine 'RagDatasetGenerator.agenerate_dataset_from_nodes' was never awaited
  obj, end = self.scan_once(s, idx)


In [21]:
from llama_index.core.indices.postprocessor import MetadataReplacementPostProcessor

postproc = MetadataReplacementPostProcessor(
    target_metadata_key="window"
)

In [22]:
query_engine = sentence_index.as_query_engine(
    similarity_top_k=2, node_postprocessors=[postproc, rerank]
)

In [23]:
response = query_engine.query('is the paper from google research?')
print(response)

Yes.


## Evaluate RAG pipeline

In [24]:
rag_evaluator_pack = RagEvaluatorPack(
    rag_dataset=eval_dataset,
    query_engine=query_engine,
    judge_llm=gpt4,
)

base_benchmark = await rag_evaluator_pack.arun(
    batch_size=10,  # batches the number of openai api calls to make
    sleep_time_in_seconds=1,  # seconds to sleep before making an api call
)


Batch processing of predictions: 100%|██████████| 2/2 [00:06<00:00,  3.01s/it]
Batch processing of evaluations:  40%|████      | 1/2.5 [00:07<00:10,  7.28s/it]


In [25]:
base_benchmark

rag,base_rag
metrics,
mean_correctness_score,4.500000
mean_relevancy_score,1.000000
mean_faithfulness_score,1.000000
mean_context_similarity_score,0.999999


## Create Functions to build RAG pipeline and Evaluation.

This will make the process of iterating easier for evaluation.

In [26]:
def build_index(documents, llm=OpenAI(model='gpt-3.5-turbo', temperature=0.1),
                embed_model="local:BAAI/bge-small-en-v1.5", sentence_window_size=3):
    
    # create the sentence window node parser w/ default settings
    node_parser = SentenceWindowNodeParser.from_defaults(
        window_size=sentence_window_size,
        window_metadata_key="window",
        original_text_metadata_key="original_text",
    )
    
    sentence_index = VectorStoreIndex.from_documents(
        documents, embed_model=embed_model, llm=llm
    )

    return sentence_index


def setup_query_engine(sentence_index, similarity_top_k=2, rerank_top_n=2, is_rerank = False):
    
    # define postprocessors
    postproc = MetadataReplacementPostProcessor(target_metadata_key="window")
    if is_rerank:
      rerank = SentenceTransformerRerank(
          top_n=rerank_top_n, model="BAAI/bge-reranker-base"
      )
      query_engine = sentence_index.as_query_engine(
          similarity_top_k=similarity_top_k, node_postprocessors=[postproc, rerank]
      )
    else:
      query_engine = sentence_index.as_query_engine(
          similarity_top_k=similarity_top_k, node_postprocessors=[postproc]
      )
    return query_engine

async def evaluate(query_engine):
  rag_evaluator_pack = RagEvaluatorPack(
      rag_dataset=eval_dataset,
      query_engine=query_engine
  )

  benchmark_df = await rag_evaluator_pack.arun(
      batch_size=10,  # batches the number of openai api calls to make
      sleep_time_in_seconds=1,  # seconds to sleep before making an api call
  )

  return benchmark_df

def build_index_query_engine(window_size,
                             similarity_top_k,
                             rerank_similarity_top_k,
                             rerank_top_k):
  sentence_index = build_index(
      [document],
      sentence_window_size=window_size,
  )

  query_engine = setup_query_engine(sentence_index,
                                      similarity_top_k=similarity_top_k)

  query_engine_rerank = setup_query_engine(sentence_index,
                                            similarity_top_k=rerank_similarity_top_k,
                                            rerank_top_n=rerank_top_k,
                                            is_rerank=True)
  return sentence_index, query_engine, query_engine_rerank

In [27]:
index, query_engine, query_engine_rerank = build_index_query_engine(1, 2, 4, 2)

In [28]:
base_benchmark = await evaluate(query_engine)

Batch processing of predictions: 100%|██████████| 2/2 [00:04<00:00,  2.33s/it]
Batch processing of evaluations:  40%|████      | 1/2.5 [00:12<00:18, 12.10s/it]


In [29]:
base_benchmark_rerank = await evaluate(query_engine_rerank)

Batch processing of predictions: 100%|██████████| 2/2 [00:06<00:00,  3.24s/it]
Batch processing of evaluations:  40%|████      | 1/2.5 [00:08<00:12,  8.63s/it]


In [30]:
base_benchmark

rag,base_rag
metrics,
mean_correctness_score,4.750000
mean_relevancy_score,1.000000
mean_faithfulness_score,1.000000
mean_context_similarity_score,0.999995


In [31]:
base_benchmark_rerank

rag,base_rag
metrics,
mean_correctness_score,4.25
mean_relevancy_score,1.00
mean_faithfulness_score,1.00
mean_context_similarity_score,1.00


From the metrics we can observe that `correctness` metric is lower (maximum 5) and including `reranker` improved metrics though it decreased the `correctness` metric.

Interesting to see there are no hallucinations as `faithfulness` metric is 1.0

## Tune parameters to make it production ready.

Let's aim to get `correctness` score of `4.5` and `relevancy` score of more than `0.9`.

### Experiment 1:

Let's increase window size and see if we can improve correctness as it gives more surrounding context.

In [ ]:
index, query_engine, query_engine_rerank = build_index_query_engine(3, 2, 4, 2)

In [ ]:
benchmark = await evaluate(query_engine)

Batch processing of predictions: 100%|██████████| 10/10 [00:04<00:00,  2.23it/s]
Batch processing of evaluations:  87%|████████▋ | 10/11.5 [01:29<00:13,  8.96s/it]


In [ ]:
benchmark_rerank = await evaluate(query_engine_rerank)

Batch processing of predictions: 100%|██████████| 10/10 [00:54<00:00,  5.50s/it]
Batch processing of evaluations:  87%|████████▋ | 10/11.5 [01:28<00:13,  8.88s/it]


In [ ]:
benchmark

rag,base_rag
metrics,
mean_correctness_score,3.775000
mean_relevancy_score,0.950000
mean_faithfulness_score,1.000000
mean_context_similarity_score,0.917831


In [ ]:
benchmark_rerank

rag,base_rag
metrics,
mean_correctness_score,3.875000
mean_relevancy_score,0.950000
mean_faithfulness_score,1.000000
mean_context_similarity_score,0.928741


The metrics did improve.

### Experiment 2:

Let'r increase `similarity_top_k` and `reranker_top_n` values and see if getting more relevant contexts improves the result.

In [ ]:
index, query_engine, query_engine_rerank = build_index_query_engine(3, 4, 8, 4)

In [ ]:
benchmark = await evaluate(query_engine)

Batch processing of predictions: 100%|██████████| 10/10 [00:05<00:00,  1.80it/s]
Batch processing of evaluations:  87%|████████▋ | 10/11.5 [01:33<00:13,  9.31s/it]


In [ ]:
benchmark_rerank = await evaluate(query_engine_rerank)

Batch processing of predictions: 100%|██████████| 10/10 [02:17<00:00, 13.78s/it]
Batch processing of evaluations:  87%|████████▋ | 10/11.5 [01:21<00:12,  8.10s/it]


In [ ]:
benchmark

rag,base_rag
metrics,
mean_correctness_score,4.100000
mean_relevancy_score,0.950000
mean_faithfulness_score,1.000000
mean_context_similarity_score,0.924449


In [ ]:
benchmark_rerank

rag,base_rag
metrics,
mean_correctness_score,4.275000
mean_relevancy_score,0.900000
mean_faithfulness_score,1.000000
mean_context_similarity_score,0.934464


We are close to our goal. We reached a `correctness` score of `4.275` and `relevancy` score of 0.9. The `context similarity score` also improved

### Experiment 3:

Let's now increase window size and see if it improves the metrics.

In [ ]:
index, query_engine, query_engine_rerank = build_index_query_engine(5, 4, 8, 4)

In [ ]:
benchmark = await evaluate(query_engine)

Batch processing of predictions: 100%|██████████| 10/10 [00:07<00:00,  1.36it/s]
Batch processing of evaluations:  87%|████████▋ | 10/11.5 [01:16<00:11,  7.67s/it]


In [ ]:
benchmark_rerank = await evaluate(query_engine_rerank)

Batch processing of predictions: 100%|██████████| 10/10 [03:14<00:00, 19.49s/it]
Batch processing of evaluations:  87%|████████▋ | 10/11.5 [01:22<00:12,  8.26s/it]


In [ ]:
benchmark

rag,base_rag
metrics,
mean_correctness_score,4.250000
mean_relevancy_score,1.000000
mean_faithfulness_score,1.000000
mean_context_similarity_score,0.926042


In [ ]:
benchmark_rerank

rag,base_rag
metrics,
mean_correctness_score,4.250000
mean_relevancy_score,1.000000
mean_faithfulness_score,1.000000
mean_context_similarity_score,0.933203


We have reached a `relevency` score of `1.0`.

### Experiment 4:

Let'r increase `similarity_top_k` and `reranker_top_n` values and see if getting more relevant contexts improves the result.

In [ ]:
index, query_engine, query_engine_rerank = build_index_query_engine(5, 6, 12, 6)

In [ ]:
benchmark_rerank = await evaluate(query_engine_rerank)

Batch processing of predictions: 100%|██████████| 10/10 [05:02<00:00, 30.20s/it]
Batch processing of evaluations:  87%|████████▋ | 10/11.5 [01:20<00:12,  8.09s/it]


In [ ]:
benchmark_rerank

rag,base_rag
metrics,
mean_correctness_score,4.50000
mean_relevancy_score,1.00000
mean_faithfulness_score,1.00000
mean_context_similarity_score,0.93266


We have reached our goal of `correctness` score of `4.5` and `relevancy` score of `1.0` (>0.9).

## Observation:

In this project, we looked into building RAG Pipeline, evaluation dataset and tuning different parameters to make it production ready. It should be observed that `reranker` improved metrics in most of the experiments.

Please do remember that we have various other metrics like `chunk_size`, `chunk_overlap`, `embedding model`, `LLM` to experiment.